<a href="https://colab.research.google.com/github/HoaNguyen55/PhoBert_News_Classification/blob/main/PhoBert_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive/')
# !mkdir /content/drive/MyDrive/News # First time create new folder
os.chdir('/content/drive/MyDrive/News/')


Mounted at /content/drive/


In [2]:
# !git clone https://github.com/HoaNguyen55/PhoBert_News_Classification.git
os.chdir('/content/drive/MyDrive/News/PhoBert_News_Classification')

In [ ]:
#@title
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM, Input, GlobalAveragePooling1D, Flatten
import tensorflow_hub as hub
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

!pip install vncorenlp
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# First time create new folder

# !mkdir -p vncorenlp/models/wordsegmenter  
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar  
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab  
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr  
# !mv VnCoreNLP-1.1.1.jar vncorenlp/   
# !mv vi-vocab vncorenlp/models/wordsegmenter/  
# !mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/  

In [ ]:
#@title
from vncorenlp import VnCoreNLP
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
from tqdm import tqdm
import pickle
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
import os
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW, RobertaTokenizer, RobertaTokenizerFast, RobertaModel, AutoTokenizer
from datetime import datetime
import glob
from sklearn.model_selection import train_test_split

classes = ['__label__sống_trẻ', 
           '__label__thời_sự', 
           '__label__công_nghệ', 
           '__label__sức_khỏe', 
           '__label__giáo_dục', 
           '__label__xe_360', 
           '__label__thời_trang', 
           '__label__du_lịch', 
           '__label__âm_nhạc', 
           '__label__xuất_bản', 
           '__label__nhịp_sống', 
           '__label__kinh_doanh', 
           '__label__pháp_luật', 
           '__label__ẩm_thực', 
           '__label__thế_giới', 
           '__label__thể_thao', 
           '__label__giải_trí', 
           '__label__phim_ảnh']

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", local_files_only=True)

def make_mask(batch_ids):
    batch_mask = []
    for ids in batch_ids:
        mask = [int(token_id > 0) for token_id in ids]
        batch_mask.append(mask)
    return torch.tensor(batch_mask)

def dataloader_from_text(text_file=None, tokenizer=None, classes=[], savetodisk=None, loadformdisk=None, segment=False, max_len=256, batch_size=16, infer=False):
    ids_padded, masks, labels = [], [], []
    if loadformdisk == None:
        #segementer
        if segment:
            rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
        texts = []
        print("LOADDING TEXT FILE")
        with open(text_file, 'r') as f_r:
            for sample in tqdm(f_r):
                if infer:
                    text = sample.strip()
                    if segment:
                        text = rdrsegmenter.tokenize(text)
                        text = ' '.join([' '.join(x) for x in text])
                    texts.append(text)
                else:
                    splits = sample.strip().split(" ",1)
                    label = classes.index(splits[0])
                    text = splits[1]
                    if segment:
                        text = rdrsegmenter.tokenize(text)
                        text = ' '.join([' '.join(x) for x in text])
                    labels.append(label)
                    texts.append(text)

        print("TEXT TO IDS")
        ids = []
        for text in tqdm(texts):
            encoded_sent = tokenizer.encode(text)
            ids.append(encoded_sent)

        del texts
        # print("PADDING IDS")
        ids_padded = pad_sequences(ids, maxlen=max_len, dtype="long", value=0, truncating="post", padding="post")
        del ids
        # print("CREATE MASK")
        # for sent in tqdm(ids_padded):
        #     masks.append(make_mask(sent))

        if savetodisk != None and not infer:
            with open(savetodisk, 'wb') as f:
                pickle.dump(ids_padded, f)
                # pickle.dump(masks, f)
                pickle.dump(labels, f)
            print("SAVED IDS DATA TO DISK")
    else:
        print("LOAD FORM DISK")
        if loadformdisk != None:
            try:
                with open(savetodisk, 'rb') as f:
                    ids_padded = pickle.load(ids_padded, f)
                    # masks = pickle.load(masks, f)
                    labels = pickle.load(labels, f)
                print("LOADED IDS DATA FORM DISK")
            except:
                print("LOAD DATA FORM DISK ERROR!")
                
    print("CONVERT TO TORCH TENSOR")
    ids_inputs = torch.tensor(ids_padded)
    del ids_padded
    # masks = torch.tensor(masks)
    if not infer:
        labels = torch.tensor(labels)

    print("CREATE DATALOADER")
    if infer:
        # input_data = TensorDataset(ids_inputs, masks)
        input_data = TensorDataset(ids_inputs)
    else:
        input_data = TensorDataset(ids_inputs, labels)
        # input_data = TensorDataset(ids_inputs, masks, labels)
    
    X_train, X_test, y_train, y_test = train_test_split(input_data, labels, test_size=0.1, shuffle=False)
    
    input_sampler_X = SequentialSampler(X_train)
    input_sampler_Y = SequentialSampler(X_test)
    train_dataloader = DataLoader(X_train, sampler=input_sampler_X, batch_size=batch_size)
    test_dataloader = DataLoader(X_test, sampler=input_sampler_Y, batch_size=batch_size)

    print("LOAD DATA ALL DONE")
    return train_dataloader, test_dataloader

def dataloader_from_text_only(text_file=None, tokenizer=None, classes=[], savetodisk=None, loadformdisk=None, segment=False, max_len=256, batch_size=16, infer=False):
    ids_padded, masks, labels = [], [], []
    if loadformdisk == None:
        #segementer
        if segment:
            rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
        texts = []
        print("LOADDING TEXT FILE")
        with open(text_file, 'r') as f_r:
            for sample in tqdm(f_r):
                if infer:
                    text = sample.strip()
                    if segment:
                        text = rdrsegmenter.tokenize(text)
                        text = ' '.join([' '.join(x) for x in text])
                    texts.append(text)
                else:
                    splits = sample.strip().split(" ",1)
                    label = classes.index(splits[0])
                    text = splits[1]
                    if segment:
                        text = rdrsegmenter.tokenize(text)
                        text = ' '.join([' '.join(x) for x in text])
                    labels.append(label)
                    texts.append(text)

        print("TEXT TO IDS")
        ids = []
        for text in tqdm(texts):
            encoded_sent = tokenizer.encode(text)
            ids.append(encoded_sent)

        del texts
        # print("PADDING IDS")
        ids_padded = pad_sequences(ids, maxlen=max_len, dtype="long", value=0, truncating="post", padding="post")
        del ids
        # print("CREATE MASK")
        # for sent in tqdm(ids_padded):
        #     masks.append(make_mask(sent))

        if savetodisk != None and not infer:
            with open(savetodisk, 'wb') as f:
                pickle.dump(ids_padded, f)
                # pickle.dump(masks, f)
                pickle.dump(labels, f)
            print("SAVED IDS DATA TO DISK")
    else:
        print("LOAD FORM DISK")
        if loadformdisk != None:
            try:
                with open(savetodisk, 'rb') as f:
                    ids_padded = pickle.load(ids_padded, f)
                    # masks = pickle.load(masks, f)
                    labels = pickle.load(labels, f)
                print("LOADED IDS DATA FORM DISK")
            except:
                print("LOAD DATA FORM DISK ERROR!")
                
    print("CONVERT TO TORCH TENSOR")
    ids_inputs = torch.tensor(ids_padded)
    del ids_padded
    # masks = torch.tensor(masks)
    if not infer:
        labels = torch.tensor(labels)

    print("CREATE DATALOADER")
    if infer:
        # input_data = TensorDataset(ids_inputs, masks)
        input_data = TensorDataset(ids_inputs)
    else:
        input_data = TensorDataset(ids_inputs, labels)
        # input_data = TensorDataset(ids_inputs, masks, labels)
        
    input_sampler = SequentialSampler(input_data)
    dataloader = DataLoader(input_data, sampler=input_sampler, batch_size=batch_size)

    print("len dataloader:", len(dataloader))
    print("LOAD DATA ALL DONE")
    return dataloader, input_data, labels

def predict_text(model, text, classes, tokenizer, max_len=256):
  model.cuda()
  ids = tokenizer.encode(text)
  ids_padded = pad_sequences([ids], maxlen=max_len, dtype="long", value=0, truncating="post", padding="post")
  mask = [[int(token_id > 0)] for token_id in ids_padded[0]]
  input_ids = torch.tensor(ids_padded)
  input_mask = torch.tensor(mask)
  with torch.no_grad():
      logits = model(input_ids.cuda(), 
                      attention_mask=input_mask.cuda(),
                      labels=None)[0]
      logits = logits.detach().cpu().numpy()
      pred_flat_axis1 = np.argmax(logits, axis=1).flatten()
      # print(pred_flat_axis1)

  print("[PREDICT] \nTEXT: {}\nLABEL: {}".format(text, classes[pred_flat_axis1[1]]))

# Validate model on new valid.txt, not relate with train.txt and test.tx file
def classification_report_model(model, file_dir, tokenizer, classes=[]):
      valid_path = file_dir
      test_dataloader, test_data_raw, test_labels = dataloader_from_text_only(valid_path, 
                                                                            tokenizer=tokenizer, 
                                                                            classes=classes, 
                                                                            savetodisk=None, 
                                                                            batch_size=32)
	    # Store label to compare with trained model and then do classification report
      true_labels = []
      model.cuda()

      for step, batch in enumerate(test_dataloader):
        b_input_ids = batch[0].cuda()
        b_input_mask = make_mask(batch[0]).cuda()
        b_labels = batch[1].cuda()
        model.zero_grad()
        with torch.no_grad():
          outputs = model(b_input_ids, 
	                      attention_mask=b_input_mask, 
	                      labels=b_labels
	                      )
          logits = outputs[1].detach().cpu().numpy()
          pred_flat = np.argmax(logits, axis=1).flatten().tolist()
          true_labels.extend(pred_flat)
       
          # print(test_labels.tolist())
          # print(true_labels)
      print(classification_report(test_labels.tolist(), true_labels, target_names=classes))

def load_model(file_dir):
  # Load model from saved file
  load_model = torch.load(file_dir)
  model = BERTClassifier(len(classes))
  model.load_state_dict(load_model['model_state_dict']) # Load state dict, include input_ids and attention_mask
  
  return model

def train_plot_model(classes, train_dataloader, test_dataloader, epochs):
  # Bert model
  bert_classifier_model = BERTClassifier(len(classes))
  # Initialize model
  bert_classifier_trainer = ClassifierTrainner(bert_model=bert_classifier_model, 
                                              train_dataloader=train_dataloader, 
                                              valid_dataloader=test_dataloader, 
                                              epochs=epochs, 
                                              cuda_device="0", # Default to use GPU, "cpu"=cpu or "0"=gpu0, "1"=gpu1, 
                                              new_file=True) # Create new folder
  # Training model 
  accur_train, loss_train, accur_val, loss_val = bert_classifier_trainer.train_classifier()

  # Plot model
  bert_classifier_trainer.plot_model(accur_train, loss_train, accur_val, loss_val)

def init_load_text(file_dir, bert_model="vinai/phobert-base", max_len=256):
  train_path = file_dir
  MAX_LEN = max_len
  tokenizer = AutoTokenizer.from_pretrained(bert_model, local_files_only=True)
  train_dataloader, test_dataloader = dataloader_from_text(train_path, 
                                                            tokenizer=tokenizer, 
                                                            classes=classes, 
                                                            savetodisk=None, 
                                                            max_len=MAX_LEN, 
                                                            batch_size=16)
  
  return train_dataloader, test_dataloader

class ROBERTAClassifier(torch.nn.Module):
    def __init__(self, num_labels, bert_model, dropout_rate=0.3):
        super(ROBERTAClassifier, self).__init__()
        if bert_model != None:
            self.roberta = bert_model
        else:
            self.roberta = RobertaModel.from_pretrained("./vinai/phobert-base")
        self.d1 = torch.nn.Dropout(dropout_rate)
        self.l1 = torch.nn.Linear(768, 64)
        self.bn1 = torch.nn.LayerNorm(64)
        self.d2 = torch.nn.Dropout(dropout_rate)
        self.l2 = torch.nn.Linear(64, num_labels)
        
    def forward(self, input_ids, attention_mask):
        _, x = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        x = self.d1(x)
        x = self.l1(x)
        x = self.bn1(x)
        x = torch.nn.Tanh()(x)
        x = self.d2(x)
        x = self.l2(x)
        return x 

class BERTClassifier(torch.nn.Module):
    def __init__(self, num_labels):
        super(BERTClassifier, self).__init__()
        bert_classifier_config = RobertaConfig.from_pretrained(
            "./vinai/phobert-base/config.json",
            from_tf=False,
            num_labels = num_labels,
            output_hidden_states=False,
            )
        print("LOAD BERT PRETRAIN MODEL")
        self.bert_classifier = RobertaForSequenceClassification.from_pretrained(
            "./vinai/phobert-base/pytorch_model.bin",
            config=bert_classifier_config
            )

    def forward(self, input_ids, attention_mask, labels):
        output = self.bert_classifier(input_ids=input_ids,
                                    token_type_ids=None,
                                    attention_mask=attention_mask,
                                    labels=labels
                                    )
        return output

class ClassifierTrainner():
    def __init__(self, bert_model, train_dataloader, valid_dataloader, epochs=10, cuda_device="cpu", save_dir=None, new_file=True):

        if cuda_device == "cpu":
            self.device == torch.device("cpu")
        else:
            self.device = torch.device('cuda:{}'.format(cuda_device))

        self.model = bert_model
        if save_dir != None and os.path.exists(save_dir):
            print("Load weight from file:{}".format(save_dir))
            self.save_dir = save_dir
            epcho_checkpoint_path = glob.glob("{}/model_epoch*".format(self.save_dir))
            if len(epcho_checkpoint_path) == 0:
                print("No checkpoint found in: {}\nCheck save_dir...".format(self.save_dir))
            else:
                self.load_checkpoint(epcho_checkpoint_path)
                print("Restore weight successful from: {}".format(epcho_checkpoint_path))
        else:
            if (new_file):
              self.save_dir = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
              os.makedirs(self.save_dir)
              print("Training new model, save to: {}".format(self.save_dir))

        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.epochs = epochs
        # self.batch_size = batch_size

    def save_checkpoint(self, save_path):
        state_dict = {'model_state_dict': self.model.state_dict()}
        torch.save(state_dict, save_path)
        print(f'Model saved to ==> {save_path}')

    def load_checkpoint(self, load_path):
        state_dict = torch.load(load_path, map_location=self.device)
        print(f'Model restored from <== {load_path}')
        self.model.load_state_dict(state_dict['model_state_dict'])

    @staticmethod    
    def flat_accuracy(preds, labels):
        pred_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        F1_score = f1_score(pred_flat, labels_flat, average='macro')
        return accuracy_score(pred_flat, labels_flat), F1_score

    def train_classifier(self):
        self.model.to(self.device)
        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
            ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False) # lr is recommended from BERT paper
        loss_train_list = []
        acurracy_train_list = []
        acurracy_val_list = []
        loss_val_list = []
        best_valid_loss = 999999
        best_eval_accuracy = 0
                
        for epoch_i in range(0, self.epochs):
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, self.epochs))
            print('Training...')
            total_loss = 0
            train_accuracy = 0
            train_f1 = 0
            nb_train_steps = 0
            self.model.train()
            
            for step, batch in enumerate(self.train_dataloader):
                b_input_ids = batch[0].to(self.device)
                b_input_mask = make_mask(batch[0]).to(self.device)
                b_labels = batch[1].to(self.device)

                self.model.zero_grad()
                outputs = self.model(b_input_ids, 
                                    attention_mask=b_input_mask, 
                                    labels=b_labels
                                    )
                loss = outputs[0]
                total_loss += loss.item()
                
                logits = outputs[1].detach().cpu().numpy()
                label_ids = b_labels.cpu().numpy()
                tmp_train_accuracy, tmp_train_f1 = self.flat_accuracy(logits, label_ids)
                train_accuracy += tmp_train_accuracy
                train_f1 += tmp_train_f1
                nb_train_steps += 1
                
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                optimizer.step()
                print("[TRAIN] Epoch {}/{} | Batch {}/{} | Train Loss={} | Train Acc={}".format(epoch_i+1, self.epochs, step, len(self.train_dataloader), loss.item(), tmp_train_accuracy))
                
                ''' In training, every step divisible by 100 or current step == max length of dataloader
                    Then validate dataset, and store the model which has the best result
                '''
                if step % 50 == 0:
                    if step > 0:
                      accur_val, loss_val = self.validate_classifier(epoch_i+1, best_valid_loss, best_eval_accuracy)
                      best_eval_accuracy = accur_val
                      best_valid_loss = loss_val
                      acurracy_val_list.append(accur_val)
                      loss_val_list.append(loss_val)

                      avg_train_loss = total_loss / len(self.train_dataloader)
                      acurracy_each_steps = train_accuracy/nb_train_steps
                      print(" Train Accuracy: {0:.4f}".format(acurracy_each_steps))
                      acurracy_train_list.append(acurracy_each_steps)
                      print(" Train F1 score: {0:.4f}".format(train_f1/nb_train_steps))
                      print(" Train Loss: {0:.4f}".format(avg_train_loss))
                      loss_train_list.append(avg_train_loss)
            
        print("Training complete!")
        return acurracy_train_list, loss_train_list, acurracy_val_list, loss_val_list

    def validate_classifier(self, currentEpoch, best_valid_loss, best_eval_accuracy):
        print("Running Validation...")
        self.model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        eval_f1 = 0
        for batch in self.valid_dataloader:
            b_input_mask = make_mask(batch[0]).to(self.device)
            batch = tuple(t.to(self.device) for t in batch)
            b_input_ids, b_labels = batch
            with torch.no_grad():
                outputs = self.model(b_input_ids, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels
                                    )
                tmp_eval_loss, logits = outputs[0], outputs[1]
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.cpu().numpy()
                tmp_eval_accuracy, tmp_eval_f1 = self.flat_accuracy(logits, label_ids)
                eval_accuracy += tmp_eval_accuracy
                eval_loss += tmp_eval_loss
                eval_f1 += tmp_eval_f1
                nb_eval_steps += 1

        loss_val = eval_loss/nb_eval_steps
        print(" Valid Loss: {0:.4f}".format(loss_val))
        accur_val = eval_accuracy/nb_eval_steps
        print(" Valid Accuracy: {0:.4f}".format(accur_val))
        print(" Valid F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))
        
        print("best_valid_loss = {0:.4f}".format(best_valid_loss))
        print("eval_loss = {0:.4f}".format(loss_val))

        if eval_loss > best_valid_loss:
            best_valid_loss_path = "{}/model_best_valoss_{}epochs.pth".format(self.save_dir, self.epochs)
            self.save_checkpoint(best_valid_loss_path)
        
        print("best_eval_accuracy = {0:.4f}".format(best_eval_accuracy))
        print("eval_accuracy = {0:.4f}".format(accur_val))

        if accur_val > best_eval_accuracy:
            best_eval_accuracy_path = "{}/model_best_valacc{}epochs.pth".format(self.save_dir, self.epochs)
            self.save_checkpoint(best_eval_accuracy_path)
        
        if currentEpoch == self.epochs:
            epoch_path = "{}/model_{}epoch.pth".format(self.save_dir, currentEpoch)
            self.save_checkpoint(epoch_path)
            # os.remove("{}/model_{}epoch.pth".format(self.save_dir, currentEpoch-1))

        return accur_val, loss_val.item()

    def plot_model(self, acurracy_train_list, loss_train_list, acurracy_val_list, loss_val_list):
      # list all data in history
      # summarize history for accuracy
      plt.plot(acurracy_train_list)
      plt.plot(acurracy_val_list)
      # formatter = matplotlib.ticker.StrMethodFormatter("{x:.3f}")
      # plt.gca().yaxis.set_major_formatter(formatter)
      plt.title('model accuracy')
      plt.ylabel('accuracy')
      plt.xlabel('epoch')
      plt.legend(['train', 'vald'], loc='upper left')
      plt.show()
      # summarize history for loss
      plt.plot(loss_train_list)
      plt.plot(loss_val_list)
      plt.title('model loss')
      plt.ylabel('loss')
      plt.xlabel('epoch*step') # Check number of step = 10 ==> 526%50 = 10
      plt.legend(['train', 'vald'], loc='upper left')
      plt.show()

    def predict_dataloader(self, dataloader, classes, tokenizer):
        for batch in dataloader:
            batch = tuple(t.to(self.device) for t in batch)
            b_input_ids, b_input_mask = batch
            with torch.no_grad():
                outputs = self.model(b_input_ids, 
                                    attention_mask=b_input_mask,
                                    labels=None
                                    )
                logits = outputs
                logits = logits.detach().cpu().numpy()
                pred_flat = np.argmax(logits, axis=1).flatten()
                print("[PREDICT] {}:{}".format(classes[int(pred_flat)], tokenizer.decode(b_input_ids)))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#@title
from huggingface_hub import hf_hub_download
# snapshot_download(repo_id="vinai/phobert-base")
hf_hub_download(repo_id="vinai/phobert-base", filename="config.json")
hf_hub_download(repo_id="vinai/phobert-base", filename="pytorch_model.bin")
hf_hub_download(repo_id="vinai/phobert-base", filename="bpe.codes")
hf_hub_download(repo_id="vinai/phobert-base", filename="tokenizer.json")
hf_hub_download(repo_id="vinai/phobert-base", filename="vocab.txt")

'/root/.cache/huggingface/hub/models--vinai--phobert-base/snapshots/667b55927a1571811539f27c0f374429a1c75759/vocab.txt'

In [ ]:
# First time create new folder

# !mkdir vinai
# !chmod 777 vinai
# !mkdir vinai/phobert-base
# !chmod 777 vinai/phobert-base
# !cp /root/.cache/huggingface/hub/models--vinai--phobert-base/snapshots/667b55927a1571811539f27c0f374429a1c75759/* vinai/phobert-base/.

In [ ]:
# Load text file
train_dataloader, test_dataloader = init_load_text(file_dir='train-1.txt')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LOADDING TEXT FILE


9335it [00:00, 37485.89it/s]


TEXT TO IDS


100%|██████████| 9335/9335 [00:25<00:00, 362.58it/s]


CONVERT TO TORCH TENSOR
CREATE DATALOADER
LOAD DATA ALL DONE


In [ ]:
# Train model and plot accuracy and loss
train_plot_model(classes, train_dataloader, test_dataloader, epochs=8)

In [ ]:
model = load_model(file_dir='./06-10-2022_09-24-11/model_8epoch.pth')

LOAD BERT PRETRAIN MODEL


Some weights of the model checkpoint at ./vinai/phobert-base/pytorch_model.bin were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./vinai/phobert-base/pytorch

In [ ]:
# Report based on validation input file
classification_report_model(model, file_dir='valid.txt', tokenizer=tokenizer, classes=classes)

LOADDING TEXT FILE


501it [00:00, 30967.73it/s]


TEXT TO IDS


100%|██████████| 501/501 [00:01<00:00, 282.34it/s]


CONVERT TO TORCH TENSOR
CREATE DATALOADER
len dataloader: 16
LOAD DATA ALL DONE
                     precision    recall  f1-score   support

  __label__sống_trẻ       1.00      1.00      1.00        25
   __label__thời_sự       0.97      1.00      0.98        31
 __label__công_nghệ       1.00      1.00      1.00        38
  __label__sức_khỏe       0.95      1.00      0.98        21
  __label__giáo_dục       1.00      0.96      0.98        25
    __label__xe_360       1.00      0.96      0.98        25
__label__thời_trang       1.00      1.00      1.00        33
   __label__du_lịch       1.00      1.00      1.00        24
   __label__âm_nhạc       1.00      1.00      1.00        40
  __label__xuất_bản       1.00      1.00      1.00        28
 __label__nhịp_sống       1.00      1.00      1.00        23
__label__kinh_doanh       1.00      1.00      1.00        19
 __label__pháp_luật       1.00      0.97      0.99        34
   __label__ẩm_thực       1.00      1.00      1.00        21
  __

In [ ]:
# Report based on train input file
classification_report_model(model, file_dir='train-1.txt', classes=classes)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LOADDING TEXT FILE


9335it [00:00, 50333.70it/s]


TEXT TO IDS


100%|██████████| 9335/9335 [00:22<00:00, 414.70it/s]


CONVERT TO TORCH TENSOR
CREATE DATALOADER
len dataloader: 292
LOAD DATA ALL DONE
                     precision    recall  f1-score   support

  __label__sống_trẻ       0.99      0.98      0.99       510
   __label__thời_sự       1.00      1.00      1.00       496
 __label__công_nghệ       1.00      0.98      0.99       532
  __label__sức_khỏe       0.98      0.99      0.99       496
  __label__giáo_dục       0.99      0.99      0.99       528
    __label__xe_360       0.99      0.99      0.99       502
__label__thời_trang       1.00      0.99      0.99       521
   __label__du_lịch       1.00      0.99      0.99       551
   __label__âm_nhạc       1.00      1.00      1.00       554
  __label__xuất_bản       0.99      1.00      0.99       519
 __label__nhịp_sống       0.97      1.00      0.99       497
__label__kinh_doanh       0.98      0.99      0.99       498
 __label__pháp_luật       1.00      0.99      1.00       543
   __label__ẩm_thực       1.00      1.00      1.00       520
  _

In [ ]:
# Demo predict a paragraph 
text = 'vì sao phim võ thuật hoa ngữ thế giới kinh ngạc phim chiếu rạp bộ phim cổ trang điện ảnh hoa ngữ thế giới kinh ngạc ngọa hổ tàng long đoạt oscar hạng mục phim nói tiếng nước ngoài xuất sắc 2001 nhắc đầu tiên bộ phim ngọa hổ tàng long đạo diễn tài ba lý an 2000 bộ phim điện ảnh châu á trở thành hiện tượng nền điện ảnh thế giới thập kỷ trôi bộ phim niềm tự hào điện ảnh trung quốc trở thành bộ phim nước ngoài doanh thu cao mỹ đồng thời bộ phim đem đạo diễn lý an loạt tượng vàng oscar 2001 vô số giải thưởng lớn nhỏ bộ phim xuất hiện pha song kiếm thuộc kinh điển tên tuổi bộ phim lý an ông đau đầu chọn địa điểm quay phù hợp phim hoàng thôn nơi thích hợp khung cảnh kiếm hiệp hấp dẫn hoàng thôn unesco ghi nhận di sản văn hóa thế giới đậm dấu ấn văn hóa trung quốc đúng ý đồ nghệ thuật đạo diễn lý an ngoài ra bộ phim góp mặt đạo diễn chỉ đạo võ thuật viên hòa bình đóng vai trò quan trọng bộ ba phim ăn khách ma trận kill bill bộ phim anh hùng bộ phim xuất sắc đạo diễn trương nghệ mưu quy tụ ngôi sao sáng màn bạc trung hoa lý liên kiệt lương triều vĩ chân tử đan trương mạn ngọc trần đạo minh chương tử di bộ phim dựa truyền thuyết kinh kha lý liên kiệt thủ vai nhân vật chính vô danh đối với anh hùng trương nghệ mưu thú nhận đầu tiên ông thực hiện bộ phim võ thuật đầy tham vọng bởi vậy đạo diễn xem xét chi tiết ông chỉ đạo đường kiếm lý liên kiệt giọt lệ lăn gò má trương mạn ngọc chính xác đường tơ kẽ tóc cảnh quay tốn ba giờ đồng hồ thường sử dụng phim giây cảnh chiến đấu trương mạn ngọc chương tử di bối cảnh diễn mùa lá rụng đạo diễn thuê vùng nội mông quay video chiếc lá gửi ông xem trương nghệ mưu theo dõi điện thoại xem lá vàng độ nào đạt yêu cầu ông đoàn phim vượt nghìn cây số tận nơi quay cảnh chiếc lá rơi tất cả mọi rời trường quay lúc quá nửa đêm thì trương nghệ mưu chỉnh sửa kết quả làm việc đạo diễn hy vọng nỗ lực mệt mỏi ông bù đắp hưởng ứng nhiệt tình khán giả bộ phim ra mắt kết quả anh hùng trương nghệ mưu lọt đề cử oscar phim nói tiếng nước ngoài xuất sắc 2002 tuy nhiên bộ phim đoạt giải người ta rằng viện hàn lâm điện ảnh hoa kỳ không bao giờ trao liên tiếp giải oscar thể loại phim võ thuật cổ trang như thế nhiếp ẩn nương the assasin tác phẩm đề tài kiếm hiệp tiếp theo đạo diễn hầu hiếu hiền trình làng bộ phim nhận lời phê bình tích cực giới phê bình giả thưởng đạo diễn xuất sắc liên hoan phim cannes 2015 thậm chí tác phẩm điện ảnh nhăm nhe tham vọng đề cử tượng vàng oscar hạng mục phim nói tiếng nước ngoài xuất sắc 2016 bộ phim khán giả cảm thấy mãn nhãn bởi thước phim xây dựng tuyệt đẹp chân thực kinh ngạc bối cảnh tuyệt đẹp bộ phim đạo diễn hầu hiếu hiền tiết lộ hầu hết cảnh quay chúng tôi thực hiện khu vực mông cổ phía bắc trung quốc tỉnh hồ bắc tôi bất ngờ nhìn khu rừng bạch dương bạc bờ hồ giữ nguyên vẻ hoang sơ bối cảnh phim tạo khuôn hình không thể đẹp đặc biệt phim khán giả bắt gặp khuôn hình nông dân làm việc mảnh đất họ nông dân thực sự công việc đúng thường ngày họ tôi cố gắng giữ nguyên tất cả cho dù vài chi tiết đúng kịch bản mong muốn'

predict_text(model, classes=classes, text=text, tokenizer=tokenizer, max_len=256)

[PREDICT] 
TEXT: vì sao phim võ thuật hoa ngữ thế giới kinh ngạc phim chiếu rạp bộ phim cổ trang điện ảnh hoa ngữ thế giới kinh ngạc ngọa hổ tàng long đoạt oscar hạng mục phim nói tiếng nước ngoài xuất sắc 2001 nhắc đầu tiên bộ phim ngọa hổ tàng long đạo diễn tài ba lý an 2000 bộ phim điện ảnh châu á trở thành hiện tượng nền điện ảnh thế giới thập kỷ trôi bộ phim niềm tự hào điện ảnh trung quốc trở thành bộ phim nước ngoài doanh thu cao mỹ đồng thời bộ phim đem đạo diễn lý an loạt tượng vàng oscar 2001 vô số giải thưởng lớn nhỏ bộ phim xuất hiện pha song kiếm thuộc kinh điển tên tuổi bộ phim lý an ông đau đầu chọn địa điểm quay phù hợp phim hoàng thôn nơi thích hợp khung cảnh kiếm hiệp hấp dẫn hoàng thôn unesco ghi nhận di sản văn hóa thế giới đậm dấu ấn văn hóa trung quốc đúng ý đồ nghệ thuật đạo diễn lý an ngoài ra bộ phim góp mặt đạo diễn chỉ đạo võ thuật viên hòa bình đóng vai trò quan trọng bộ ba phim ăn khách ma trận kill bill bộ phim anh hùng bộ phim xuất sắc đạo diễn trương ngh

In [4]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 7.5 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=31d606720ecd3a2c7f0cb3874392307be9410a490acadcdcb5eddf3b0cf27e49
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [233]:
import docx
# doc = docx.Document('TESU CBE 29 Employee Job Description Evaluation - Final Approved.docx')  # Creating word reader object.
doc = docx.Document('FAIR2021_submissions_2022-10-10_1665416741.docx')
data = ""
fullText = []
# for para in doc.paragraphs:
#     print(para)
#     fullText.append(para.text)
#     data = '\n'.join(fullText)

# print(data)
print(len(doc.paragraphs))
for i in range(len(doc.paragraphs)):
  print(doc.paragraphs[i].text)
  if (i>20):
    break


399

Submission list
Paper index: [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [24], [25], [27], [29], [30], [32], [33], [34], [35], [38], [42], [43], [46], [48], [49], [51], [52], [54], [59], [61], [64], [65], [66], [68], [72], [73], [74], [78], [79], [82], [83], [84], [85], [86], [87], [89], [90], [91], [92], [93], [95], [96], [98], [101], [102], [105], [106], [108], [111], [113], [115], [117], [118], [119], [121], [123], [124], [125], [126], [127], [128], [129], [130], [131], [132], [133], [134], [135], [136], [139], [140], [141], [142], [143], [144], [145], [146], [147], [148], [149], [150], [151], [152], [154], [155], [157], [158], [159], [160], [161], [162], [163], [164], [165], [166], [189], [190], [191], [192], [193]
[1] Brijeshkumar Y. Panchal (charotar university of science and technology, India.) and Gaurang Chauhan (ITsouls, Vdodara). Design and implementation of android application to extract text from

In [333]:
import re

def extract_abstract(doc):
  abstractLst = []
  prefix = ["Abstract.", 
            "Tóm Tắt -", 
            "Tóm Tắt-", 
            "TÓM TẮT -", 
            "TÓM TẮT—",
            "Tóm tắt -",
            "TÓM TẮT —",
            "Abstract -",
            "Abstract-",
            "—"]
  for i in range(len(doc.paragraphs)):
    match = re.search('(?<!\S)Abstract[\.].+[[a-zA-Z0-9a-zA-Z_ÀÁÂÃÈÉÊẾÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêếìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ].+', doc.paragraphs[i].text)        
    if match != None:
      tmp_str = match.group(0)
      for idx in prefix:
        tmp_str = tmp_str.replace(idx, "")
      tmp_str = tmp_str.strip()
      abstractLst.append(tmp_str)
    
  return abstractLst

def extract_author_title(doc):
  authorLst = []
  titleLst = []
  for i in range(len(doc.paragraphs)):
    # Use for English only
    # match = re.search('\[[0-9]+\]\s+[a-zA-Z0-9].+', doc.paragraphs[i].text)
    # Use for Vietnamese and English
    match = re.search('^\[[0-9]+\]+\s[a-zA-Z0-9a-zA-Z_ÀÁÂÃÈÉÊẾÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêếìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳỵỷỹ].+', doc.paragraphs[i].text) # Use for Vietnamese
    # result = (doc.paragraphs[i].text).startswith('[')
    if match:
      str_list = match.group(0)
      author_list_split = str_list.split(". ")
      author_list_split = list(filter(str.strip, author_list_split))
      name_author = " ".join(author_list_split[:-1])
      name_author = name_author.split()[1:]
      name_author = " ".join(name_author)

      name_article = author_list_split[-1]
      
      authorLst.append(name_author)
      titleLst.append(name_article.strip())
    
  return authorLst, titleLst

def extract_index(doc):
  indexLst = []
  for i in range(len(doc.paragraphs)):
    match = re.search(r'(?<!\S)Paper index:\s+(\[[A-Za-z0-9].+)', doc.paragraphs[i].text)

    if match:
        idxLst = match.group(1).split(",")
        for idx in idxLst:
          idxStrip = idx.strip(" []")
          indexLst.append(int(idxStrip))

  return indexLst

def extract_keywords(doc):
  keywordLst = []
  for i in range(len(doc.paragraphs)):
    keywords = re.findall("Keywords:.+", doc.paragraphs[i].text)
    if (len(keywords) > 0):
      tmp_str = ' '.join(keywords)
      # Remove "Keywords:" word
      prefix = ["Keywords:"]
      for pre in prefix:
        tmp_str = tmp_str[tmp_str.startswith(pre) and len(pre):]
        tmp_str = " ".join(tmp_str.split())

      keywordLst.append(tmp_str)
    
  return keywordLst

def extract_track(doc):
  trackLst = []
  for i in range(len(doc.paragraphs)):
    track = re.findall("Track:.+", doc.paragraphs[i].text)
    if (len(track) > 0):
      tmp_str = ' '.join(track)
      # Remove "Track:" word
      prefix = ["Track:"]
      for pre in prefix:
        tmp_str = tmp_str[tmp_str.startswith(pre) and len(pre):]
        tmp_str = " ".join(tmp_str.split())
      trackLst.append(tmp_str)
    
  return trackLst

def export_csv(index, track, author, title, keywords, abstract, folderName=None):
  import pandas as pd

  #make pandas df and store in runs folder
  all_records = []

  for i in range(len(author)):
    all_records.append({'Index':index[i], 'Author':author[i], 'Title':title[i], 'Keywords':keywords[i], 'Track':track[i], 'Abstract':abstract[i]})

  if folderName != None:
    name_of_run = folderName
  else:
    name_of_run = 'fair2021_dataset'
  
  df_long   = pd.DataFrame(all_records, columns=['Index', 'Author', 'Title', 'Keywords', 'Track', 'Abstract'])
  csv_name  = './' + 'fair2021_dataset.csv'
  df_long[['Index', 'Author', 'Title', 'Keywords', 'Track', 'Abstract']].to_csv(csv_name, index=False)
  

In [334]:
index = extract_index(doc)
track = extract_track(doc)
keywords = extract_keywords(doc)
author, title = extract_author_title(doc)
abstract = extract_abstract(doc)
print(len(index))
print(len(track))
print(len(keywords))
print(len(author))
print(len(title))
print(len(abstract))
# export_csv(index, track, author, title, keywords, abstract)

121
121
121
121
121
121


In [335]:
export_csv(index, track, author, title, keywords, abstract)